In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%matplotlib inline

/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# concatenate data file

In [ ]:
USER_DISCTIONARY = {}
data_root = '/home/ai2-jedi/Downloads/Geolife Trajectories 1.3/Data/'

In [ ]:
#concatenate gps censor data
daily_gps = pd.DataFrame(columns=['userID','lat','long','zeros','alt','GMT','date','time'])
for user in os.listdir(data_root):
    print user
    for f in os.listdir(data_root + user + '/Trajectory'):
        try:
            each_plt=pd.DataFrame(np.genfromtxt(data_root + user + '/Trajectory/'+str(f), delimiter=',',skip_header=6),\
                                 columns=['lat','long','zeros','alt','GMT','date','time'])
            each_plt['userID']=user
            each_plt=each_plt.round({'lat': 5, 'long':5}) #reduce lat-long to 5 decimal points
            each_plt['GMT']=each_plt['GMT'].map(lambda x: convert_time(x))
            daily_gps=daily_gps.append(each_plt)
        except:
            pass
#daily_gps.reset_index()    
print len(daily_gps)

In [ ]:
#Beijing GeoFence
upper_left_latlong=(40.195024, 115.891955)
lower_right_latlong=(39.676959, 116.850512)

In [ ]:
Beijing_GPS=daily_gps[(daily_gps['lat']>39.676959) & (daily_gps['lat']<40.195024)& (daily_gps['long']>115.891955)& (daily_gps['long']<116.850512)]

In [ ]:
#Beijing_GPS.to_csv('Geolife_Beijing_raw.csv')

In [88]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    mile = 6367 * c *0.621371
    return mile

In [ ]:
def calculate_distance(oneUser):
    oneUser[['lat','long','previousLat','previousLong']] = oneUser[['lat','long','previousLat','previousLong']].apply(pd.to_numeric)
    oneUser['hdist'] = oneUser.apply(lambda x: haversine_np(x[4], x[3],x[8], x[7]), axis = 1)
    #oneUser['deltaHour']=oneUser['deltaTime'].apply(lambda x:(x/(np.timedelta64(1, 'm'))/60.0))
    #oneUser['speed(mile/hour)']=oneUser['hdist']/oneUser['deltaHour']
    return oneUser

In [ ]:
def convert_time(delta):
    temp = datetime.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    time = temp+datetime.timedelta(days=delta)
    return time

In [ ]:
def convert_string_to_time(time):
    time = datetime.datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f')
    #time=time.strftime('%Y/%m/%d %H:%M:%S')
    return time

In [ ]:
def convert_string_to_time_slash(time):
    time = datetime.datetime.strptime(str(time),'%Y/%m/%d %H:%M:%S')
    #time=time.strftime('%Y/%m/%d %H:%M:%S')
    return time

In [ ]:
def reformat_time(time):
    #time = datetime.datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f')
    time=time.strftime('%Y/%m/%d %H:%M:%S')
    return time

In [ ]:
def findTimeDiff(df):
    df=df.sort('GMT',ascending=True)
    df['time2']=df['GMT'].shift(-1)
    df['deltaTime'] = df['time2']-df['GMT']
    
    df['previousLat'] = df['lat'].shift(-1)
    df['previousLong'] = df['long'].shift(-1)
    

    df=calculate_distance(df)
    
    df['deltaHour']=df['deltaTime'].apply(lambda x:(x/(np.timedelta64(1, 'm'))/60.0))
    df['speed(mile/hour)']=df['hdist']/df['deltaHour']

    return df[['userID','GMT','lat','long','speed(mile/hour)']]

In [ ]:
Beijing_GPS['GMT']=Beijing_GPS['GMT'].apply(lambda x:convert_string_to_time(x))

In [ ]:
UserList=list(set(Beijing_GPS['userID']))

In [ ]:
speed_df=pd.DataFrame(columns=['userID','GMT','lat','long','speed(mile/hour)'])
for user in UserList:
    single_user_df=Beijing_GPS[Beijing_GPS['userID']==user]
    single_user_df=findTimeDiff(single_user_df)
    speed_df=speed_df.append(single_user_df)
speed_df.to_csv('Beijing_Geolife_w_speed.csv')

In [ ]:
#labels table
gps_label = pd.DataFrame(columns=['userID','Start Time','End Time','Transportation Mode'])
for user in os.listdir(data_root):
    try:
        each_txt=pd.read_csv(data_root + user +'/labels.txt', delimiter='\t')
        each_txt['userID']=user
        gps_label=gps_label.append(each_txt)
        #print gps_label
    except:
        pass

In [ ]:
speed_df_timeReformat=speed_df.copy()
gps_label_timeReformat=gps_label.copy()
#speed_df_timeReformat['GMT']=speed_df_timeReformat['GMT'].apply(lambda x: reformat_time(x))

In [ ]:
gps_label_timeReformat[['userID']] = gps_label_timeReformat[['userID']].apply(pd.to_numeric)
gps_label_timeReformat['Start Time']=gps_label_timeReformat['Start Time'].apply(lambda x: convert_string_to_time_slash(x))
gps_label_timeReformat['End Time']=gps_label_timeReformat['End Time'].apply(lambda x: convert_string_to_time_slash(x))
gps_label_timeReformat.head()

In [ ]:
#speed_df_timeReformat.to_csv('Geolife_Beijing_w_Speed.csv')

# reduce dimension

In [3]:
speed_df_timeReformat=pd.read_csv('Geolife_Beijing_w_Speed.csv')

In [4]:
speed_df_timeReformat=speed_df_timeReformat.dropna()

In [5]:
speed_df_timeReformat.groupby('userID').count().describe()

,Unnamed: 0,GMT,lat,long,speed(mile/hour)
count,115.000000,115.000000,115.000000,115.000000,115.000000
mean,98249.156522,98249.156522,98249.156522,98249.156522,98249.156522
std,198367.933732,198367.933732,198367.933732,198367.933732,198367.933732
min,14.000000,14.000000,14.000000,14.000000,14.000000
25%,2163.000000,2163.000000,2163.000000,2163.000000,2163.000000
50%,16501.000000,16501.000000,16501.000000,16501.000000,16501.000000
75%,113075.000000,113075.000000,113075.000000,113075.000000,113075.000000
max,1516872.000000,1516872.000000,1516872.000000,1516872.000000,1516872.000000


In [6]:
def sample_gps(df,sample_rate):
    # next, sample every nth row (where n=sample_rate) of the full gps location history data set
    df_sampled = df.iloc[range(0, len(df), sample_rate)]
    return df_sampled

In [7]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#centermost_points = clusters[:21].map(get_centermost_point)

In [53]:
def get_centroid_list(clusters):
    centroidList=[]
    for each_cluster in clusters:
        try:
            centroidList.append(get_centermost_point(each_cluster))
        except:
            pass
    return centroidList

In [50]:
def multiple_user_cluster(df, eps, x='long', y='lat'):
    eps=eps
    UserList=list(set(df['userID']))
    total_clusters_all_users=pd.DataFrame(columns=['cLat', 'cLon', 'userID'])
    for user in UserList:
        print user
        sub_df=df[df['userID']==user]
        sample_rate=len(sub_df)/20000
        if sample_rate!=0:
            df_sample=sample_gps(sub_df,sample_rate)
        else:
            df_sample=sub_df
        sub_clustered=dbscan_reduce(user,df_sample, epsilon=eps, x='long', y='lat')
        total_clusters_all_users=total_clusters_all_users.append(sub_clustered)
    return total_clusters_all_users

In [52]:
def dbscan_reduce(user,df_original, epsilon, x='long', y='lat'):
    df=df_original.copy()
    start_time = time.time()
    # represent points consistently as (lat, lon) and convert to radians to fit using haversine metric
    coords = df.as_matrix(columns=[y, x])    
    db = DBSCAN(eps=epsilon, min_samples=1000, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    #print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters_w_empty = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    clusters=[cluster for cluster in clusters_w_empty if cluster!=[]]
    # find the point in each cluster that is closest to its centroid
    #centermost_points = clusters.map(get_centermost_point)
    centermost_points = get_centroid_list(clusters)
    centroidTable=pd.DataFrame(centermost_points,columns=['cLat','cLon'])
    centroidTable['userID']=user
    #return centermost_points
    return centroidTable

In [76]:
def dbscan_reduce_totalUser(df_original, epsilon, x='cLon', y='cLat'):
    df=df_original.copy()
    # represent points consistently as (lat, lon) and convert to radians to fit using haversine metric
    coords = df.as_matrix(columns=[y, x])    
    db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    #print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters_w_empty = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    clusters=[cluster for cluster in clusters_w_empty if cluster!=[]]
    # find the point in each cluster that is closest to its centroid
    #centermost_points = clusters.map(get_centermost_point)
    centermost_points = get_centroid_list(clusters)
    centroidTable=pd.DataFrame(centermost_points,columns=['cLat','cLon'])
    #return centermost_points
    return centroidTable

In [18]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088
# first cluster the full gps location history data set coarsely, with epsilon=5km in radians
eps_rad = 1 / kms_per_radian
#df_clustered = multiple_user_cluster(speed_df_timeReformat[:1000000], epsilon=eps_rad)

In [59]:
cluster_by_user=multiple_user_cluster(df=speed_df_timeReformat,eps=eps_rad)

0.0
1.0
2.0
4.0
6.0
7.0
8.0
9.0
10.0
11.0
17.0
18.0
20.0
21.0
24.0
25.0
26.0
27.0
29.0
30.0
31.0
33.0
36.0
37.0
38.0
39.0
42.0
46.0
47.0
48.0
50.0
51.0
52.0
53.0
56.0
57.0
58.0
60.0
61.0
64.0
65.0
69.0
72.0
74.0
75.0
76.0
77.0
78.0
79.0
81.0
82.0
84.0
85.0
86.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
99.0
100.0
101.0
105.0
107.0
110.0
111.0
113.0
114.0
115.0
116.0
117.0
120.0
121.0
122.0
123.0
126.0
127.0
129.0
130.0
131.0
133.0
134.0
135.0
136.0
137.0
138.0
140.0
141.0
142.0
143.0
146.0
147.0
150.0
152.0
153.0
154.0
155.0
156.0
159.0
161.0
162.0
164.0
166.0
167.0
168.0
170.0
171.0
174.0
177.0
178.0
179.0
181.0


/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


In [61]:
cluster_by_user.to_csv('cluster_by_user.csv',index=False)

In [79]:
total_cluster=dbscan_reduce_totalUser(cluster_by_user,epsilon=eps_rad)

/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


In [80]:
total_cluster.to_csv('total_cluster.csv',index=False)

# Single User Analysis

In [81]:
user85=speed_df_timeReformat[speed_df_timeReformat['userID']==85].sort('GMT',ascending=True)

/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [96]:
firstFew_user85=user85

In [98]:
def return_closest_centroid(lat,lon,centroidTable):
    distance_dict=[]
    for cIndex in centroidTable.index:
        #print pd.DataFrame([haversine_np(lon, lat, centroidTable.iloc[cIndex][1], centroidTable.iloc[cIndex][0]),cIndex]).transpose()
        distance_dict.append([haversine_np(lon, lat, total_cluster.iloc[cIndex][1], total_cluster.iloc[cIndex][0]),cIndex])
        #print distance_dict
    distance_dict=pd.DataFrame(distance_dict)
    distance_dict.columns=['distance','cIndex']
    distance_dict.loc[distance_dict.distance >5 , 'cIndex'] = 800
    distance_sort=distance_dict.sort('distance',ascending=True)
    return distance_sort.iloc[0][1]

In [99]:
clusterList=[]
for row in firstFew_user85.index:
    lat=firstFew_user85['lat'][row]
    lon=firstFew_user85['long'][row]
    cluster=return_closest_centroid(lat,lon,total_cluster)
    clusterList.append(cluster)
firstFew_user85['cluster']=clusterList

/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [100]:
for i in range(8):
    print len(firstFew_user85[firstFew_user85['cluster']==i])

253584
124930
30901
42848
9909
13845
77017
0


In [101]:
firstFew_user85.to_csv('user85_allClusterTag.csv')